In [ ]:
def replacemany(our_str, to_be_replaced:str, replace_with:str):
    while (to_be_replaced in our_str):
        our_str = our_str.replace(to_be_replaced, replace_with)
    return our_str

def _filter_dict(argsstr):
    if not argsstr or len(argsstr.strip())<1:
        return None
    env_dict={}
    argsstr=replacemany(replacemany(replacemany(argsstr.strip(),('  '),' '),('= '),'='),' =','=')
    envstr=str(str(argsstr.split(" ")).split("=")).replace(" ","").replace("\'","").replace("\"","").replace("[","").replace("]","").replace("\\","")
    env_list=envstr.split(",")
    for i in range(0,len(env_list),2):
        env_dict[str(env_list[i])]=env_list[i+1]
    return env_dict

envstr=' a1  =     1  b2=3     y3=6  '
print(replacemany(replacemany(replacemany(envstr.strip(),('  '),' '),('= '),'='),' =','='))
print(_filter_dict(envstr))


In [ ]:
from urllib.request import urlopen

myURL = urlopen("")
print(myURL.read())

In [ ]:
## %runinterm
##%term:cmd /c start 
##%overwritefile
##%file:src/test.py
##%stdin<-:myfifo
##%cleartest
##%log:0
import sys
print('Plase input your name: ')
name = sys.stdin.readline()
print('Hello {0}'.format(name))

##test_begin
print("------中文测试------")
print("------测试代码------\n")
##test_end

In [ ]:
import mmap
import contextlib
import time
import atexit
import platform
import tempfile
import os
from queue import Queue

fcntl = None
msvcrt = None
bLinux = True
if platform.system() != 'Windows':
    fcntl = __import__("fcntl")
    bLinux = True
else:
    msvcrt = __import__('msvcrt')
    bLinux = False
##
##
##
class CFileLock:
    def __init__(self, filename):
        # self.kobj=kobj
        # self.tmpdir = tempfile.TemporaryDirectory()
        self.tmpdir =tempfile.gettempdir()
        fname=filename+ '.lock'
        fname = os.path.join(self.tmpdir, fname)
        self.filename = fname
        self.file = None
    def __del__(self):
        pass
        # if os.path.exists(self.filename):
            # os.remove(self.filename)
        # self.tmpdir.cleanup()
    def lock(self):
        if bLinux is True:
            self.file = open(self.filename, 'wb')
            try:
                fcntl.flock(self.file, fcntl.LOCK_EX | fcntl.LOCK_NB)
            except:
                return False
        else:
            self.file = open(self.filename, 'wb')
            try:
                msvcrt.locking(self.file.fileno(), msvcrt.LK_LOCK, 1)
            except Exception as e:
                return False
        return True
 
    def unlock(self):
        try:
            if bLinux is True:
                fcntl.flock(self.file, fcntl.LOCK_UN)
                self.file.close()
            else:
                self.file.seek(0)
                msvcrt.locking(self.file.fileno(), msvcrt.LK_UNLCK, 1)
        except Exception as e:
            pass
def read_all_from_queue(queue):
    res = b''
    size = queue.qsize()
    while size != 0:
        res += queue.get_nowait()
        size -= 1
    return res
i=0
flock=CFileLock('myfifo')
_queue = Queue()
cmdstart_time = time.time()
to=180
while i<75:
    s=b''
    flag=b''
    with contextlib.closing(mmap.mmap(-1, 4096, tagname='myfifo', access=mmap.ACCESS_DEFAULT)) as m:
        
        if flock.lock():
            m.seek(0)
            flag=m.read(1)
            ## 检查flag是否为1，1为可读
            if flag!=b'\x01':
                # print("wait...")
                time.sleep(1/1000)
                run_time = time.time() - cmdstart_time
                if run_time > to:break
                continue
            m.seek(0)
            m.write(b'\x02')
            m.flush()
            m.seek(1)
            s = m.read()
            m.seek(0)
            m.write(b'\x00'*4096)
            m.flush()
            i+=1
            time.sleep(1/1000)
            # flock.unlock()
            s1=s.decode().strip(b'\x00'.decode())
            if len(s1)>0:
                print(s1.strip()+str(i))
            flock.unlock()
            time.sleep(1/1000)
#   _contents = read_all_from_queue(_queue)
#   s1=_contents.decode('UTF-8', errors='ignore').strip(b'\x00'.decode())
#   if len(s1)>0:
#     print(s1.strip())


In [8]:
import re
 
line = "//%Cat:s are smarter than dogs"
 
matchObj = re.match( r'(.*) are (.*?) .*', line)
matchObj1= re.search( r':(.*)',line)
if matchObj1:
    print ("matchObj1.group() : "+ matchObj1.group(0))
else:
   print ("No match!!")

value="cmd dd -g -o asdf"

args=[]
for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
    args += [argument.strip('"')]
print(args.pop(0) +"\n")
print(args[0] +"\n")

#for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
    #print(argument)

[MyPythonKernel125536] Info:The process :h:\Jupyter\ipynb\MyPython\tmp2g9zpjum.py
[MyPythonKernel125536] Info:python3 h:\Jupyter\ipynb\MyPython\tmp2g9zpjum.py 


matchObj1.group() : :s are smarter than dogs
cmd

dd



[MyPythonKernel125536] Info:The process end:12616


In [ ]:
##%overwritefile
##%file:src/test1.py
import multiprocessing
import os
# 读数据进程执行的代码:
def _read(q):
    print('Process(%s) is reading...' % os.getpid())
    while True:
        url = q.get(True)
        print('Get %s from queue.' % url)
def main():
    # 1. 创建一个队列 (先进先出)
    q = multiprocessing.Queue(10) # 最多放10个数据。 如果不指定长度,默认最大(和硬件相关)
    # 2. 创建多个进程，将队列的引用当做实参进行传递
    p1 = multiprocessing.Process(target=_read, args=(q,))
    p1.start()

# main1() 
if __name__ == "__main__":
    # multiprocessing.freeze_support()
    main() 

In [ ]:
import time
import asyncio

now = lambda : time.time()

async def do_some_work(x):
    print('Waiting: ', x)
    return 'Done after {}s'.format(x)

def callback(future):
    print('Callback: ', future.result())

start = now()

coroutine = do_some_work(2)
print("this is test")
loop = asyncio.get_event_loop()
task = loop.create_task(coroutine)
#task = asyncio.ensure_future(coroutine)	# 貌似和上面用loop创建任务效果一样
task.add_done_callback(callback)
loop.run_until_complete(task)


In [ ]:

import zerorpc

class HelloRPC(object):
    def hello(self, name):
        return "Hello, %s" % name

s = zerorpc.Server(HelloRPC())
s.bind("tcp://0.0.0.0:4249")
s.run()

In [ ]:
##%srvmode: tcp://0.0.0.0:4255
##%rpcsrvfollowcode
import sys
print('Plase input your name: ')
name = sys.stdin.readline()
print('Hello {0}'.format(name))

In [ ]:
import zerorpc

class HelloRPC(object):
    def hello(self, name):
        return "Hello, %s" % name

s = zerorpc.Server(HelloRPC())
s.bind("ipc:///tmp/feeds/0")
s.run()

In [ ]:
##%assfile:MyPython2.ipynb 8
##%assfile:..\..\Myjupyter-kernel\Mythink.ipynb 4
print("this is test code.")

In [7]:
##%log:1
print("this is test code.")

[MyPythonKernel125536] Info:The process :h:\Jupyter\ipynb\MyPython\tmp4oj6cg90.py
[MyPythonKernel125536] Info:python3 h:\Jupyter\ipynb\MyPython\tmp4oj6cg90.py 


this is test code.


[MyPythonKernel125536] Info:The process end:10640
